In [14]:
import pandas as pd

In [20]:
df = pd.read_csv('train.csv')

df.info(verbose=True)

df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'train.csv'

In [ ]:
#Separating numerics and non numerics. This method is necessary because we often treat them with different methods

numerics_col = df.select_dtypes(include=['number']).columns
print(numerics_col)

non_numerics_col = df.select_dtypes(exclude=['number']).columns
print(non_numerics_col)

Cleaning - Missing Data

In [ ]:
#Checking for the missing data

num_missing = df.isna().sum()
num_missing[:10]


In [ ]:
#Checking the percentage missing data

pct_missing = df.isna().mean()
pct_missing[:10]

In [ ]:
# Checking missing data with heatmap. Because it only works when you have smaller datasets, we will only use the first 30 columns of our datasets

# Using 2 Python libraries to create the heatmap: seaborn and missingno.

import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10,8))

cols = df.columns[:30]
colours = ['#000099','#ffff00'] #yellow - missing, blue - not missing
sns.heatmap(df[cols].isna(), cmap=sns.color_palette(colours))

In [ ]:
# Checking missing data (by rows) with histogram


missing_by_row = df.isna().sum(axis='columns')
missing_by_row.hist(bins=50)

In [ ]:
# Cleaning with missing data. Drop column with over 30% missing data

pct_missing[pct_missing > .3]
df_less_missing_cols = df.drop(columns=pct_missing[pct_missing > .3].index)
df_less_missing_cols.shape

In [ ]:
# Cleaning with missing data. Drop row to only keep with less than 35 missing columns.

df_less_missing_rows = df[missing_by_row < 35].copy()
df_less_missing_rows.shape

In [ ]:
# Cleaning data with imputing constant value

df_copy = df.copy()
df_copy[numerics_col] = df_copy[numerics_col].fillna(-999)
df_copy[non_numerics_col] = df_copy[non_numerics_col].fillna('_MISSING_')

In [ ]:
# Cleaning data with imputing statistics value

df_copy = df.copy()
med = df_copy[numerics_col].median()
df_copy[numerics_col] = df_copy[numerics_col].fillna(med)

#Imputing non numeric columns with their most frequent value
most_freq = df_copy[non_numerics_col].describe().loc['top']
df_copy[non_numerics_col] = df_copy[non_numerics_col].fillna(most_freq)

Cleaning Data - Outliers

In [ ]:
# Detect potential outliers

df.kurt(numeric_only=True)[:10] #life_sq has the highest number of kurtosis


In [ ]:
# look at the column’s common descriptive statistics
df['life_sq'].describe()

In [ ]:
# Let’s use the data visualization method to detect outliers

df['life_sq'].hist(bins=100)

In [ ]:
df.boxplot(column=['life_sq'])

In [ ]:
# Using bar chart to learn about outliers in non numerical data
df['ecology'].value_counts().plot(kind='bar')

Cleaning Data - Unnecessary Data
We consider the data to be unnecessary when it doesn’t add value. 

In [ ]:
# Check for repetitive & uninformative value
# Show columns with over 99.9% rows being the same value

num_rows = len(df)
for col in df.columns:
    cnts = df[col].value_counts(dropna=False)
    top_percentage = (cnts/num_rows).iloc[0]

    if top_percentage > 0.999:
        print('{0}: {1:.2f}%'.format(col, top_pct*100))
        print(cnts)
        print()


In [ ]:
# Check for duplicates value in rows
df[df.duplicated()]

In [ ]:
# Check for duplicates value in columns
df[df.drop(columns=['id']).duplicated()]

In [ ]:
#drop the duplicate value
df_dedupped = df.drop(columns=['id']).drop_duplicates()
print (df_dedupped.shape)

In [ ]:

# For house sales, let’s assume that if two transactions have the same timestamp, 
# full_sq, life_sq, floor, build_year, num_room, price_doc, they are duplicates. 

key = ['timestamp', 'full_sq', 'life_sq', 'floor', 'build_year', 'num_room', 'price_doc']

df_grouped = df.fillna(-999).groupby(key)['id'].count()

df_grouped[df_grouped > 1]

In [ ]:
# Drop duplicates

df_dedupped2 = df.drop_duplicates(subset=key)

print(df_dedupped.shape)
print(df_dedupped2.shape)

Cleaning Data - Inconsistent Data

In [ ]:
# cleaning capitalization because python is case sensitive
df['sub_area'].value_counts(dropna=False)
df['sub_area_lower'] = df['sub_area'].str.lower()
df['sub_area_lower'].value_counts(dropna=False)

In [ ]:
# cleaning inconsistent data types

df['timestamp']

df['timestamp_dt'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%d')
df['year'] = df['timestamp_dt'].dt.year
df['month'] = df['timestamp_dt'].dt.month
df['weekday'] = df['timestamp_dt'].dt.weekday

df[['timestamp_dt', 'year', 'month', 'weekday']].head()

In [ ]:
df_city_ex = pd.DataFrame(data={'city': ['torontoo', 'toronto', 'tronto', 'vancouver', 'vancover', 'vancouvr', 'montreal', 'calgary']})

cities = ['toronto', 'vancouver', 'montreal', 'calgary']
from nltk.metrics import edit_distance
for city in cities:
    df_city_ex[f'city_distance_{city}'] = df_city_ex['city'].map(lambda x: edit_distance(x, city))

df_city_ex

In [ ]:
# Set criteria to convert these typos to the correct values.

msk = df_city_ex['city_distance_toronto'] <= 2
df_city_ex.loc[msk, 'city'] = 'toronto'

msk = df_city_ex['city_distance_vancouver'] <= 2
df_city_ex.loc[msk, 'city'] = 'vancouver'

df_city_ex

There is no address column in our housing dataset due to privacy. Let’s create a new dataset df_add_ex with a column address.

In [ ]:
df_add_ex = pd.DataFrame(['123 MAIN St Apartment 15', '123 Main Street Apt 12   ', '543 FirSt Av', '  876 FIRst Ave.'], columns=['address'])
df_add_ex

We can run the below code to:

- lowercase the letters
- remove leading and trailing white spaces
- delete periods
- standardize wordings

In [16]:
df_add_ex['address_std'] = df_add_ex['address'].str.lower()
df_add_ex['address_std'] = df_add_ex['address_std'].str.strip() # remove leading and trailing whitespaces.
df_add_ex['address_std'] = df_add_ex['address_std'].str.replace('\\.', '', regex=True) # remove period.
df_add_ex['address_std'] = df_add_ex['address_std'].str.replace('\\bstreet\\b', 'st', regex=True) # replace street with st.
df_add_ex['address_std'] = df_add_ex['address_std'].str.replace('\\bapartment\\b', 'apt', regex=True) # replace apartment with apt.
df_add_ex['address_std'] = df_add_ex['address_std'].str.replace('\\bav\\b', 'ave', regex=True) # replace av with ave.

df_add_ex

,address,address_std
0,123 MAIN St Apartment 15,123 main st apt 15
1,123 Main Street Apt 12,123 main st apt 12
2,543 FirSt Av,543 first ave
3,876 FIRst Ave.,876 first ave
